In [1]:
# this cell is tagged parameters

PYLIB_DIR = None

########################
# inputs for quant-only
########################

# Reference info
REF_gtf_file = None
REF_quant_file = None

# Predictions
FLAMES_gtf_file = None
FLAMES_quant_file = None

IsoQuant_quant_file = None

IsoSeq_gtf_file = None
IsoSeq_quant_file = None

LRAA_quant_file = None

Mandalorion_gtf_file = None
Mandalorion_quant_file = None

Oarfish_align_quant_file = None

Oarfish_reads_quant_file = None

Bambu_quant_file = None

ESPRESSO_quant_file = None

FLAIR_quant_file = None

Isosceles_gtf_file = None
Isosceles_quant_file = None

StringTie_quant_file = None

TALON_gtf_file = None
TALON_quant_file = None

In [ ]:
import sys, os, re
sys.path.insert(0, PYLIB_DIR)


In [ ]:
import BenchmarkingRoutines
from importlib import reload
reload(BenchmarkingRoutines)
from BenchmarkingRoutines import *

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# colors for plots
set_color_palette("FLAMES", "gainsboro", "solid")
set_color_palette("IsoQuant", "blue", "solid")
set_color_palette("IsoSeq", "orchid", "solid")
set_color_palette("LRAA", "teal", "solid")
set_color_palette("Mandalorion", "lightblue", "solid")
set_color_palette("Oarfish_align", "khaki", "solid")
set_color_palette("Oarfish_reads", "peachpuff", "solid")
set_color_palette("Bambu", "forestgreen", "solid")
set_color_palette("ESPRESSO", "brown", "solid")
set_color_palette("FLAIR", "pink", "solid")
set_color_palette("Isosceles", "red", "solid")
set_color_palette("StringTie", "aquamarine", "solid")
set_color_palette("TALON", "orange", "solid")


In [ ]:

i_ref_df = indexDfByIntronId(parseGTFtoIntronIDsandQuants(REF_gtf_file, REF_quant_file))
i_ref_df

In [ ]:
 i_ref_df["tpm"] = i_ref_df["tpm"] / i_ref_df["tpm"].sum() * 1e6

In [ ]:
i_ref_df.copy().reset_index().to_csv("refDf.intron_ids_and_expression.tsv", sep="\t", index=False)

In [ ]:
quant_only_dir = "processed_prog_results"

prog_quant_files = {  
    "FLAMES" : [FLAMES_quant_file, FLAMES_gtf_file],
    "IsoQuant" : [IsoQuant_quant_file, REF_gtf_file],
    "IsoSeq" : [IsoSeq_quant_file, IsoSeq_gtf_file],
    "LRAA" : [LRAA_quant_file, REF_gtf_file ],
    "Mandalorion" : [Mandalorion_quant_file, Mandalorion_gtf_file],
    "Oarfish_align" : [Oarfish_align_quant_file, REF_gtf_file],
    "Oarfish_reads" : [Oarfish_reads_quant_file, REF_gtf_file],
    "ESPRESSO" : [ESPRESSO_quant_file, REF_gtf_file],
    "FLAIR" : [FLAIR_quant_file, REF_gtf_file],
    "Isosceles" : [Isosceles_quant_file, Isosceles_gtf_file],
    "Bambu" : [Bambu_quant_file, REF_gtf_file],
    "StringTie" : [StringTie_quant_file, REF_gtf_file],
    "TALON" : [TALON_quant_file, TALON_gtf_file]
}


fullQuantsDf_dict = {}
for progname, (tsv_fname, gtf_fname) in prog_quant_files.items():
    if tsv_fname is None:
        continue

    print(progname, tsv_fname, gtf_fname)
    fullQuantsDf_dict[progname] = indexDfByIntronId(parseGTFtoIntronIDsandQuants(gtf_fname, tsv_fname))


progname_to_i_sample_df_dict_to_tsv(fullQuantsDf_dict, "progname_to_IntronId_expr_vals.tsv")

In [ ]:
scatterplot_adj(i_ref_df, fullQuantsDf_dict)

In [ ]:
ma_plot_adj(i_ref_df, fullQuantsDf_dict)

In [ ]:
spearman_df = cor_spearman_barplot(i_ref_df, fullQuantsDf_dict)
spearman_df.to_csv("spearman_expr_cor.tsv", sep="\t", quoting=csv.QUOTE_NONE)

In [ ]:
pearson_df = cor_pearson_barplot(i_ref_df, fullQuantsDf_dict)
pearson_df.to_csv("pearson_expr_cor.tsv", sep="\t", quoting=csv.QUOTE_NONE)

In [ ]:
median_rel_diff_df = rel_diff_barplot(i_ref_df, fullQuantsDf_dict, 'median')
median_rel_diff_df.to_csv("median_rel_diff.tsv", sep="\t", quoting=csv.QUOTE_NONE)

In [ ]:
mean_rel_diff_df = rel_diff_barplot(i_ref_df, fullQuantsDf_dict, 'mean')
mean_rel_diff_df.to_csv("mean_rel_diff.tsv", sep="\t", quoting=csv.QUOTE_NONE)

In [ ]:
rel_diff_vs_expr_percentile_plot(i_ref_df, fullQuantsDf_dict, 33, 'median',
                                 'all ref-reduced sets, all ref transcripts')